In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, random_split
import tqdm
import pandas as pd
import copy
import numpy as np
import multiprocessing as mp
from sklearn.metrics import roc_auc_score, confusion_matrix

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

Device: 'cuda'


In [ ]:
model_name = "mlpnorm"
year = 2019
month = 11
perc = 0

x_train = torch.load(f"ds/mlp{year}{month}{perc}x_train.pt")
x_test = torch.load(f"ds/mlp{year}{month}{perc}x_test.pt")
y_train = torch.load(f"ds/mlp{year}{month}{perc}y_train.pt")
y_test = torch.load(f"ds/mlp{year}{month}{perc}y_test.pt")

print(f"x_train shape:", x_train.shape)
print(f"x_test shape:", x_test.shape)
print(f"y_train shape:", y_train.shape)
print(f"y_test shape:", y_test.shape)

/tmp/ipykernel_33107/1128881189.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_train = torch.load(f"ds/mlp{year}{month}{perc}x_train.pt")


x_train shape: torch.Size([863004, 85])
x_test shape: torch.Size([50736, 85])
y_train shape: torch.Size([863004])
y_test shape: torch.Size([50736])


/tmp/ipykernel_33107/1128881189.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_test = torch.load(f"ds/mlp{year}{month}{perc}x_test.pt")
/tmp/ipykernel_33107/112888118

In [4]:
def apply_normalization(x_train, x_test):
    mean = torch.mean(x_train, dim=0)
    std = torch.std(x_train, dim=0)

    x_train_normalized = (x_train - mean) / std
    x_test_normalized = (x_test - mean) / std

    return x_train_normalized, x_test_normalized

In [5]:
def apply_pca_explained_variance_normalized(x_train, x_test, target_variance=0.9):
    # Calculate mean and standard deviation from training data
    x_train_normalized, x_test_normalized = apply_normalization(x_train, x_test)

    # Perform PCA with all components initially
    U, S, V = torch.pca_lowrank(x_train_normalized)

    # Calculate explained variance ratio
    explained_variance_ratio = S**2 / torch.sum(S**2)
    cumulative_variance_ratio = torch.cumsum(explained_variance_ratio, dim=0)

    # Find the number of components to explain target variance
    n_components = torch.argmax((cumulative_variance_ratio >= target_variance).int()) + 1

    # Project the normalized data onto the selected components
    x_train_pca = torch.matmul(x_train_normalized, V[:, :n_components])
    x_test_pca = torch.matmul(x_test_normalized, V[:, :n_components])

    return x_train_pca, x_test_pca, n_components

In [ ]:
apply_norm = True

if apply_norm:
    index_mask = [0, 1, 2, 8, *[i for i in range(15, 42)]]
    index_mask += [i + 42 for i in index_mask] + [84]
    keep_indexes = [i for i in range(x_train.shape[1]) if i not in index_mask]
    x_train_norm, x_test_norm = apply_normalization(x_train[:, index_mask], x_test[:, index_mask])

    new_x_train = torch.hstack([x_train_norm, x_train[:, keep_indexes]])
    new_x_test = torch.hstack([x_test_norm, x_test[:, keep_indexes]])

    x_train = new_x_train
    x_test = new_x_test

    print(new_x_train.shape)
    print(new_x_test.shape)

In [ ]:
apply_PCA = False

if apply_PCA:
    index_mask = [0, 1, 2, 8, *[i for i in range(15, 42)]]
    index_mask += [i + 42 for i in index_mask] + [84]
    keep_indexes = [i for i in range(x_train.shape[1]) if i not in index_mask]
    x_train_pca, x_test_pca, num_components_used = apply_pca_explained_variance_normalized(x_train[:, index_mask], x_test[:, index_mask], target_variance=0.9)

    new_x_train = torch.hstack([x_train_pca, x_train[:, keep_indexes]])
    new_x_test = torch.hstack([x_test_pca, x_test[:, keep_indexes]])

    x_train = new_x_train
    x_test = new_x_test

    print(f"Number of components used: {num_components_used}")
    print(new_x_train.shape)
    print(new_x_test.shape)

Number of components used: 5
torch.Size([863004, 27])
torch.Size([50736, 27])


In [8]:
results_df = pd.read_csv("results.csv", dtype={
    "model": str,
    "year": int,
    "month": int,
    "perc": float,
    "epoch": int,
    "train_loss": float,
    "val_loss": float,
    "acc": float,
    "prec": float,
    "rec": float,
    "f1": float,
    "auc": float,
    "tp": int,
    "fp": int,
    "fn": int,
    "tn": int,
    "best_threshold": float,
    "done": bool
})

filtered_df = results_df[
    (results_df["model"] == model_name) &
    (results_df["year"] == year) &
    (results_df["month"] == month) &
    (results_df["perc"] == perc)
]

if filtered_df.empty:
    latest_epoch = 0
    is_trained = False
else:
    latest_epoch = filtered_df["epoch"].max()
    is_trained = filtered_df["done"].any()

print("Latest epoch:", latest_epoch)
print("Is trained?", is_trained)

Latest epoch: 0
Is trained? False


In [9]:
input_size = x_train.shape[1]

In [10]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

In [11]:
train_size = int(0.8 * x_train.shape[0])
val_size = x_train.shape[0] - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=10, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, num_workers=10, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, num_workers=10, pin_memory=True)

In [13]:
class MLP(nn.Module):
    def __init__(self, input_size: int):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
        # self.layers = nn.Sequential(
        #     nn.Linear(input_size, 128),
        #     nn.ReLU(),
        #     nn.Linear(128, 1),
        # )

    def forward(self, x):
        return self.layers(x)

In [14]:
def calculate_metrics(threshold, all_probs, all_labels):
    preds_binary = (all_probs > threshold).astype(int)
    cm = confusion_matrix(all_labels, preds_binary)
    tp = cm[1, 1]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tn = cm[0, 0]
    precision = 0 if tp == 0 else tp / (tp + fp)
    recall = 0 if tp == 0 else tp / (tp + fn)
    f1 = 0 if precision * recall == 0 else 2 * precision * recall / (precision + recall)
    return threshold, f1

In [15]:
def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs, results_df, patience=5):
    best_threshold = 0.0
    best_val_f1 = 0.0
    epochs_no_improve = 0
    best_model_state = None
    train_losses = list()
    val_losses = list()
    best_epoch = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        model.train()  # Set model to training mode
        train_loss = 0.0
        for inputs, targets in tqdm.tqdm(train_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, targets)
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update the weights

            train_loss += loss.item()

        train_loss /= len(train_loader)
        print("Train loss:", train_loss)
    
        # Validation
        model.eval()
        val_loss = 0.0
        all_labels = []
        all_probs = []  # Store probabilities for ROC-AUC
        print("Validating...")
        with torch.no_grad():
            for inputs, targets in tqdm.tqdm(val_loader):
                inputs = inputs.to(device)
                targets = targets.to(device)

                outputs = model(inputs).squeeze(1)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

                # Get predictions and probabilities (assuming binary classification with sigmoid output)
                probs = torch.sigmoid(outputs).cpu().numpy()  # Apply sigmoid if needed
                labels = targets.cpu().numpy()

                all_labels.extend(labels)
                all_probs.extend(probs.flatten())

        val_loss /= len(val_loader)

        # Find threshold for predictions
        print("Looking for threshold")

        with mp.Pool(10) as pool:
            results = pool.starmap(
                calculate_metrics, 
                [
                    (threshold, all_probs, all_labels)
                    for threshold in np.arange(0.05, 0.96, 0.01)
                ]
            )

        best_threshold_epoch = 0
        best_f1_epoch = -1
        for threshold, f1 in results:
            if f1 > best_f1_epoch:
                best_f1_epoch = f1
                best_threshold_epoch = threshold

        print(f"Best threshold: {best_threshold_epoch}")
        all_preds = (all_probs > best_threshold_epoch).astype(int)

        cm = confusion_matrix(all_labels, all_preds)
        tp = cm[1, 1]
        fp = cm[0, 1]
        fn = cm[1, 0]
        tn = cm[0, 0]

        accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0.0 # Handle division by zero
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        roc_auc = roc_auc_score(all_labels, all_probs)

        print(f"Validation Metrics - Epoch {epoch+1}/{num_epochs}:")
        print(f"Loss      :{val_loss:.4f}")
        print(f"Accuracy:  {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall:    {recall:.4f}")
        print(f"F1-score:  {f1:.4f}")
        print(f"ROC-AUC:   {roc_auc:.4f}")
        print(f"Confusion Matrix:\n{tp} {fn}\n{fp} {tn}")

        new_row = pd.DataFrame(
            {
                "model": [model_name],
                "year": [year],
                "month": [month],
                "perc": [perc],
                "epoch": [latest_epoch + epoch + 1],
                "train_loss": [train_loss],
                "val_loss": [val_loss],
                "acc": [accuracy],
                "prec": [precision],
                "rec": [recall],
                "f1": [f1],
                "auc": [roc_auc],
                "tp": [tp],
                "fp": [fp],
                "fn": [fn],
                "tn": [tn],
                "best_threshold": [best_threshold_epoch],
                "done": [False]
            }
        )
        results_df = pd.concat([results_df, new_row], ignore_index=True)
        results_df.to_csv("results.csv", index=False)

        torch.save(model.state_dict(), f"./model_{model_name}_{year}_{month}_{perc}_{latest_epoch + epoch + 1}.pth")

        if f1 > best_val_f1:
            best_val_f1 = f1
            best_threshold = best_threshold_epoch
            epochs_no_improve = 0
            best_model_state = copy.deepcopy(model.state_dict())
            best_epoch = latest_epoch + epoch + 1
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping!!!")
                print(f"Early stopping!!!")
                print(f"Early stopping!!!")
                print("Best epoch:", best_epoch)
                model.load_state_dict(best_model_state)
                break
    
    return best_threshold


In [16]:
def test(model, test_loader, device, criterion, best_threshold):
    model.eval()
    test_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    with torch.no_grad():
        for inputs, targets in tqdm.tqdm(test_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs).squeeze(1)
            loss = criterion(outputs, targets)  # Use criterion here
            test_loss += loss.item()

            probs = torch.sigmoid(outputs).cpu().numpy()  # Apply sigmoid if needed
            preds = (probs > best_threshold).astype(int)  # Convert probabilities to predictions
            labels = targets.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds.flatten())
            all_probs.extend(probs.flatten())

    test_loss /= len(test_loader)

    cm = confusion_matrix(all_labels, all_preds)
    tp = cm[1, 1]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tn = cm[0, 0]

    accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
    try:
      roc_auc = roc_auc_score(all_labels, all_probs)
    except ValueError:
        roc_auc = 0.0

    print(f"Test Metrics:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    print(f"Confusion Matrix:\n{tp} {fn}\n{fp} {tn}")
    print(f"Test Loss: {test_loss:.4f}") # Print the loss as well


In [17]:
model = MLP(input_size).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

best_threshold = train(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    device,
    1000,
    results_df
)

Epoch 1/1000


100%|██████████| 5394/5394 [00:08<00:00, 607.47it/s]


Train loss: 0.3675663983259415
Validating...


100%|██████████| 1349/1349 [00:01<00:00, 944.50it/s] 

Looking for threshold


...

100%|██████████| 5394/5394 [00:09<00:00, 591.44it/s]


Train loss: 0.30015916174535273
Validating...


100%|██████████| 1349/1349 [00:01<00:00, 890.11it/s] 

Looking for threshold


Best threshold: 0.43000000000000005
Validation Metrics - Epoch 33/1000:
Loss      :0.3097
Accuracy:  0.8639
Precision: 0.8557
Recall:    0.8753
F1-score:  0.8654
ROC-AUC:   0.9402
Confusion Matrix:
75524 10760
12737 73580
Early stopping!!!
Early stopping!!!
Early stopping!!!
Best epoch: 28


In [18]:
test(
    model,
    test_loader,
    device,
    criterion,
    best_threshold
)

100%|██████████| 397/397 [00:00<00:00, 544.88it/s]


Test Metrics:
Accuracy:  0.8327
Precision: 0.8241
Recall:    0.8460
F1-score:  0.8349
ROC-AUC:   0.9183
Confusion Matrix:
21461 3907
4581 20787
Test Loss: 0.3609
